In [36]:
import import_ipynb
import IonisationCrossSections as ICS
import DiscModel as DM
import LossFunctions as LF
import DiscModelLossFunctions as DMLF
import FileWritingReading as FWR
import GeneralFunctions as GF
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

In [37]:

def Integrate_Primary_Flux_times_Differential_Cross_Section(E,E_array,PrimaryFlux,Ionization_Energy,DifferentialIonizationFunction):
    differentialcross=[]
    for  e in E_array:
        if e > Ionization_Energy+E:
            differentialcross.append(DifferentialIonizationFunction(E, e))
            
        else: 
            differentialcross.append(0)
        
    return integrate.trapezoid(differentialcross*PrimaryFlux,E_array)


In [38]:

def Average_Differential_Cross_Section_Electron(E,E_array,PrimaryFlux,fH,fH2,fHe):
    
    I_H=Integrate_Primary_Flux_times_Differential_Cross_Section(E,E_array,PrimaryFlux,GF.Ionization_Energy_H,ICS.Differential_Ionisation_Cross_Section_Electron_H)
    I_H2=Integrate_Primary_Flux_times_Differential_Cross_Section(E,E_array,PrimaryFlux,GF.Ionization_Energy_H2,ICS.Differential_Ionisation_Cross_Section_Electron_H2)
    I_He=Integrate_Primary_Flux_times_Differential_Cross_Section(E,E_array,PrimaryFlux,GF.Ionization_Energy_He,ICS.Differential_Ionisation_Cross_Section_Electron_He)

    return fH * I_H + fHe * I_He + fH2 * I_H2
            
def Average_Differential_Cross_Section_Proton(E,E_array,PrimaryFlux,fH,fH2,fHe):
    
    I_H=Integrate_Primary_Flux_times_Differential_Cross_Section(E,E_array,PrimaryFlux,GF.Ionization_Energy_H,ICS.Differential_Ionisation_Cross_Section_Proton_H)
    I_H2=Integrate_Primary_Flux_times_Differential_Cross_Section(E,E_array,PrimaryFlux,GF.Ionization_Energy_H2,ICS.Differential_Ionisation_Cross_Section_Proton_H2)
    I_He=Integrate_Primary_Flux_times_Differential_Cross_Section(E,E_array,PrimaryFlux,GF.Ionization_Energy_He,ICS.Differential_Ionisation_Cross_Section_Proton_He)

    return fH * I_H + fHe * I_He + fH2 * I_H2
  


def Secondary_Electron_Flux_Electron(E_array,DiscModelName,Tflare,i):
    
    Primary_Flux=FWR.Read_Propagated_Flux_from_Text_File("Propagated_Spectra_Proton",DiscModelName,Tflare)[1]

    DiscModel=DM.readDiscModel(DiscModelName) 
    LossFunction2DArray=DMLF.Loss_Function_Electron_2DArray(E_array, i, DiscModel)

    fH=DM.Relative_Density_H(DiscModel)[i][::-1]
    fHe=DM.Relative_Density_He(DiscModel)[i][::-1]
    fH2=DM.Relative_Density_H2(DiscModel)[i][::-1]
    Primary_Flux[Primary_Flux < 0] = 0
    Secondary_Flux = []
    for j in range(len(LossFunction2DArray)):
        Secondary_Flux_Interm = []
        k=0
        for E in E_array:
            Integral_Secondary_Flux = Average_Differential_Cross_Section_Electron(E,E_array,Primary_Flux[j],fH[j],fH2[j],fHe[j])
            
            Local_Loss_Function= (LossFunction2DArray[::-1][j])[k]
            
            secondary_flux_value = E / Local_Loss_Function * Integral_Secondary_Flux
            
            Secondary_Flux_Interm.append(secondary_flux_value)
            k += 1
        Secondary_Flux.append(Secondary_Flux_Interm)
    FWR.Write_Propagated_Flux_to_Text_File('Secondary_Electron_Flux_Electron',Secondary_Flux,DiscModelName,i,Tflare)
    return np.array(Secondary_Flux)


def Secondary_Electron_Flux_Proton(E_array, DiscModelName,Tflare, i):
    Primary_Flux=FWR.Read_Propagated_Flux_from_Text_File("Propagated_Spectra_Proton",DiscModelName,Tflare)[1]

    DiscModel=DM.readDiscModel(DiscModelName) 

    LossFunction2DArray=DMLF.Loss_Function_Proton_2DArray(E_array, i, DiscModel)
    fH=DM.Relative_Density_H(DiscModel)[i][::-1]
    fHe=DM.Relative_Density_He(DiscModel)[i][::-1]
    fH2=DM.Relative_Density_H2(DiscModel)[i][::-1]
    Primary_Flux[Primary_Flux < 0] = 0
    Secondary_Flux = []
    for j in range(len(LossFunction2DArray)):
        Secondary_Flux_Interm = []
        k=0
        for E in E_array:
            Integral_Secondary_Flux = Average_Differential_Cross_Section_Proton(E,E_array,Primary_Flux[j],fH[j],fH2[j],fHe[j])
            
            Local_Loss_Function= (LossFunction2DArray[::-1][j])[k]
            
            secondary_flux_value = E / Local_Loss_Function * Integral_Secondary_Flux
            
            Secondary_Flux_Interm.append(secondary_flux_value)
            k += 1
        Secondary_Flux.append(Secondary_Flux_Interm)
    FWR.Write_Propagated_Flux_to_Text_File('Secondary_Electron_Flux_Proton',Secondary_Flux,DiscModelName,i,Tflare)
    return np.array(Secondary_Flux)


In [39]:

#Secondary_Flux_Electron=Secondary_Electron_Flux_Electron(E_array,Primary_Flux_Electron,LossFunctionElectron2D,DiscModel,i)
#Secondary_Flux_Proton=Secondary_Electron_Flux_Proton(E_array,Primary_Flux_Proton,LossFunctionProton2D,DiscModel,i)

In [40]:
'''
def plot_Particle_Flux(Propagated_Flux_Particle1, Propagated_Flux_Particle2, DiscModel, i):
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow']  # Define a list of colors
    i=0
    for j in range(len(Propagated_Flux_Particle1)):
        if (j+1) % 4 == 0:
            logN = np.log10((DM.ColumnDensityArray(DiscModel)[i])[::-1][j])
            if logN > 19:
                if logN < 25.2:
                    color = colors[i % len(colors)]  # Select color based on j
                    plt.plot(E_array, Propagated_Flux_Particle1[j], '--', color=color)
                    plt.plot(E_array, Propagated_Flux_Particle2[j], color=color, label=str(round(logN, 2)))
                    i+=1

    #plt.plot(E_array, Propagated_Flux_Particle1[0], color='black', label='Injection')
    #plt.plot(E_array, Propagated_Flux_Particle2[0], color='red', label='Injection')

    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Energy [eV]')
    plt.ylabel('j[part/eV/cm^2/s/sr]')
    plt.legend()
    plt.show()

    
    

plot_Particle_Flux(Tertiary_Flux_Electron,Secondary_Flux_Electron,DiscModel,i)

#plot_Particle_Flux('Electron',Primary_Flux_Electron,DiscModel,i)
'''

"\ndef plot_Particle_Flux(Propagated_Flux_Particle1, Propagated_Flux_Particle2, DiscModel, i):\n    colors = ['blue', 'red', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow']  # Define a list of colors\n    i=0\n    for j in range(len(Propagated_Flux_Particle1)):\n        if (j+1) % 4 == 0:\n            logN = np.log10((DM.ColumnDensityArray(DiscModel)[i])[::-1][j])\n            if logN > 19:\n                if logN < 25.2:\n                    color = colors[i % len(colors)]  # Select color based on j\n                    plt.plot(E_array, Propagated_Flux_Particle1[j], '--', color=color)\n                    plt.plot(E_array, Propagated_Flux_Particle2[j], color=color, label=str(round(logN, 2)))\n                    i+=1\n\n    #plt.plot(E_array, Propagated_Flux_Particle1[0], color='black', label='Injection')\n    #plt.plot(E_array, Propagated_Flux_Particle2[0], color='red', label='Injection')\n\n    plt.xscale('log')\n    plt.yscale('log')\n    plt.xlabel('Energy [eV]')\n    